## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

c:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedInProfile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
kendal.garrett@outlook.com
www.linkedin.com/in/kendal-
g-1a212aa3 (LinkedIn)
Top Skills
Microsoft Azure
Apache Kafka
Quantitative Research
Certifications
Matillion ETL Foundations Certified
Hands On Essentials - Data
Warehouse (Fundamentals)
SnowPro Core Certification
Microsoft Certified Solutions Expert:
Data Management and Analytics
Certification
Cyber Security Awareness Version
2.0
Kendal Garrett
Director of Analytics Services
Greater Chicago Area
Experience
Pandata Group
3 years 10 months
Director of Analytics Services
March 2025 - Present (5 months)
Madison, Wisconsin, United States
Analytics Practice Lead
August 2024 - March 2025 (8 months)
Madison, Wisconsin, United States
- Lead BI development, overseeing client dashboards, reporting, and database
migration
projects
- Research & Development for GTM Solutions & Strategy Implementation
- Served as PM role during contracts, structuring timelines, sprint, and stories
related to epics •
- Developed BI reporting practice 

In [2]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Kendal Garrett"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Kendal Garrett. You are answering questions on Kendal Garrett's website, particularly questions related to Kendal Garrett's career, background, skills and experience. Your responsibility is to represent Kendal Garrett for interactions on the website as faithfully as possible. You are given a summary of Kendal Garrett's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Kendal Garrett. I'm Analytics Expert, passionate ceramicist, loyal friend, and hard worker. \nI'm someone who has has years of experience leadings teams, with clear outcomes/objectives as well as making hard decisions for the greater good of the team and company.\nWith over 10 years in the Business Intelligence/Analytics space I've dedicated my career to building databases, driving insights through visualiz

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
import os

groq_api_key = os.getenv('GROQ_API_KEY')
groqapi = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

In [15]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = groqapi.beta.chat.completions.parse(model=model_name, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
print(messages)
print(response)
reply

[{'role': 'system', 'content': "You are acting as Kendal Garrett. You are answering questions on Kendal Garrett's website, particularly questions related to Kendal Garrett's career, background, skills and experience. Your responsibility is to represent Kendal Garrett for interactions on the website as faithfully as possible. You are given a summary of Kendal Garrett's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Kendal Garrett. I'm Analytics Expert, passionate ceramicist, loyal friend, and hard worker. \nI'm someone who has has years of experience leadings teams, with clear outcomes/objectives as well as making hard decisions for the greater good of the team and company.\nWith over 10 years in the Business Intelligence/Analytics space I've dedicated my career to building databases, dr

'No, I do not hold a patent. My expertise lies primarily in analytics, business intelligence, and integrating AI into BI projects. If you have other questions regarding my experience or skills, feel free to ask!'

In [18]:
import json

def evaluate(reply, message, history):
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user",   "content": evaluator_user_prompt(reply, message, history)},
    ]
    # 1.) Use .create()
    response = groqapi.beta.chat.completions.parse(
        model="llama-3.3-70b-versatile",
        messages=messages,
        temperature=2
    )
    # 2.) Extract the raw assistant text
    raw = response.choices[0].message.content
    # 3.) Parse it yourself
    return print(raw)

evaluate(reply, "do you hold a patent?", messages[:1])


**acceptable**

The response is concise and to the point, directly addressing the user's question about whether Kendal Garrett holds a patent. It also reiterates his areas of expertise and invites further questions, which is good for engagement. The tone is professional and polite, suitable for a conversation on a personal website. 

One potential area for improvement is providing more context about why he doesn't hold a patent. Is it because he hasn't had the opportunity, or is it not a priority in his field? However, given the straightforward nature of the question, the response is adequate and acceptable as is.


In [19]:
# 1. Get feedback on the original reply
feedback = evaluate(reply, "do you hold a patent?", messages[:1])

# 2. Define a rerun function with explicit parameter names
def rerun(reply, system_prompt, history, feedback):
    # Build an updated system prompt that includes the rejection reason
    updated_system_prompt = system_prompt + "\n\n" \
        "## Previous answer rejected\n" \
        "Your last response did not pass quality control.\n\n" \
        "## Your attempted answer:\n" \
        f"{reply}\n\n" \
        "## Reason for rejection:\n" \
        f"{feedback}\n\n"
    
    # Assemble the full message list
    all_messages = (
        [{"role": "system", "content": updated_system_prompt}]
        + history
        + [{"role": "user", "content": "do you hold a patent?"}]
    )
    
    # Call the model again
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=all_messages
    )
    return response.choices[0].message.content

# 3. Invoke rerun with the proper arguments
new_reply = rerun(
    reply=reply,
    system_prompt=system_prompt,
    history=messages,
    feedback=feedback
)

print(new_reply)


The response is acceptable. 

The Agent's response directly answers the user's question in a clear and concise manner. It also stays in character as Kendal Garrett, citing their expertise in analytics, business intelligence, and AI integration, which is consistent with the provided context. Additionally, the response invites the user to ask further questions, showing a willingness to engage and provide more information, which is suitable for a professional like Kendal Garrett. 

One minor suggestion for improvement could be to rephrase the last sentence to be more proactive, for example, "I'd be happy to discuss my experience and skills further, please let me know what specific areas you're interested in learning more about." This would demonstrate an even greater level of enthusiasm for interacting with potential clients or employers. However, the current response is still professional, concise, and effective in addressing the user's query.
No, I do not hold a patent. My expertise lie

In [21]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be factual- \
              spend a little more time reviewing the linkedin profile and resume to ensure your statements are accurates."
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    new_reply =response.choices[0].message.content

    evaluation = evaluate(new_reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
        new_reply = rerun(new_reply, message, response, evaluation.feedback)       
        
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        new_reply = rerun(new_reply, message, response, evaluation.feedback)       
    return new_reply
 

In [22]:
print(new_reply)

No, I do not hold a patent. My expertise lies primarily in analytics, business intelligence, and integrating AI into BI projects. If you have any other questions regarding my experience or skills, feel free to ask!


In [23]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
